In [2]:
import torch.nn as nn

encoder_layer = nn.TransformerEncoderLayer(d_model=768, nhead=8)
model = nn.TransformerEncoder(encoder_layer, num_layers=2)

/home/mputo/miniconda3/envs/deeplearning/lib/python3.11/site-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [3]:
from transformers import AutoTokenizer, AutoModel
import torch

# load
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert = AutoModel.from_pretrained("bert-base-uncased")

# tokenize
text = ["hello world", "this is a sentence"]
batch = tokenizer(text, return_tensors="pt", padding=True)

# get embeddings
with torch.no_grad():
    x = bert.embeddings.word_embeddings(batch["input_ids"])  # [B, S, 768]


/home/mputo/miniconda3/envs/deeplearning/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import torch.nn as nn

class TransformerWithClassifier(nn.Module):
    def __init__(self, encoder, embed_dim=256, num_classes=2):
        super().__init__()
        self.encoder = encoder
        self.classifier = nn.Linear(embed_dim, num_classes)

    def forward(self, embedded_inputs, attention_mask=None):
        # Get pooled embedding from your transformer
        vec = self.encoder(embedded_inputs)   # [B, embed_dim]

        # Classify
        logits = self.classifier(vec)         # [B, num_classes]
        return logits


In [5]:
classifier = TransformerWithClassifier(model, embed_dim=768, num_classes=2)

torch.mean(classifier.forward(x), dim=1).shape

torch.Size([2, 2])

In [6]:
from datasets import load_dataset

en = load_dataset("allenai/c4", "en", streaming=True)

In [7]:
import random
from transformers import pipeline

generator = pipeline('text-generation', model='gpt2')

def get_text():
    for elem in en["train"]:
        if random.choice([0, 1]) == 0:
            yield [elem["text"], 0]
        else:
            words = random.randint(1, 10)
            length = min([len(elem["text"]), 64])
            yield [generator(elem["text"][:words], max_length=length)[0]['generated_text'], 1]


Device set to use cpu


In [8]:
criterion = nn.CrossEntropyLoss()
optimiezer = torch.optim.Adam(classifier.parameters(), lr=1e-4)

for text, classification in get_text():
    optimiezer.zero_grad()

    data = [text]
    batch = tokenizer(data, return_tensors="pt", padding=True)
    # get embeddings
    with torch.no_grad():
        x = bert.embeddings.word_embeddings(batch["input_ids"])  # [B, S, 768]

    logits = torch.mean(classifier.forward(x), dim=1) # [B, num_classes]

    loss = criterion(logits, torch.tensor(classification).unsqueeze(0))
    loss.backward()

    print(f"Loss: {loss.item()}")

    optimiezer.step()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Loss: 0.7269925475120544
Loss: 1.2731863260269165
Loss: 0.7004156112670898


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Loss: 1.8251268863677979
Loss: 0.22014634311199188
Loss: 0.1661280393600464
Loss: 0.11332400888204575


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Loss: 1.5714027881622314
Loss: 2.042088508605957
Loss: 0.1328207403421402


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Loss: 0.13374504446983337


KeyboardInterrupt: 

In [11]:
data = ["BBQ classes in Missoula! Come and learn the art of grilling."]
batch = tokenizer(data, return_tensors="pt", padding=True)
# get embeddings
with torch.no_grad():
    x = bert.embeddings.word_embeddings(batch["input_ids"])  # [B, S, 768]

logits = torch.mean(classifier.forward(x), dim=1) # [B, num_classes]

logits

tensor([[0.6662, 0.0920]], grad_fn=<MeanBackward1>)